# Day 7: No Space Left on Device

## Part 1

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

```
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
```

The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

- cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
    - cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
    - cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
    - cd / switches the current directory to the outermost directory, /.
- ls means list. It prints out all of the files and directories immediately contained by the current directory:
    - 123 abc means that the current directory contains a file named abc with size 123.
    - dir xyz means that the current directory contains a directory named xyz.

Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

```
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
```

Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

- The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
- The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
- Directory d has total size 24933642.
- As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [1]:
# Loading the terminal commands from the input file
with open('input.txt', 'r') as f:
    terminal_commands = f.read().splitlines()

In [2]:
# Instantiating a list that will hold all the directories in our filesystem
filesystem = []

# Instantiating a dictionary that will hold all individual filenames and their file sizes
files = {}

# Iterating through all the terminal commands
for command in terminal_commands:
    
    # Setting the current working directory (cwd) back to root
    if command == '$ cd /':

        # Instantiating the current working directory
        cwd = ['/']
        filesystem.append('/')
    
    # Going up one directory
    elif command == '$ cd ..':
        
        # Removing the last position from cwd
        cwd.pop()
        
    # Changing the directory appropriately
    elif command.startswith('$ cd'):
        
        # Getting the directory name
        dir_name = command.split(' ')[-1]
        
        # Adding the new directory to the end of cwd
        cwd.append(dir_name + '/')
        
        # Conjoining the cwd as a single string
        cwd_joined = ''.join(cwd)
        
        # Adding the folder to the filesystem
        filesystem.append(cwd_joined)
        
    # Adding the size of the file to the dir_size
    elif command[0].isdigit():
        
        # Splitting the command between file size and filename
        file_size, filename = command.split(' ')
        
        # Converting the file_size from a string to an integer
        file_size = int(file_size)
        
        # Conjoining the cwd as a single string
        cwd_joined = ''.join(cwd)
        
        # Creating a name to represent the full filepath
        full_filepath = f'{cwd_joined}/{filename}'
        
        # Setting the directory size in our filesystem
        files[full_filepath] = file_size

In [3]:
# Instantiating a dictionary to hold the directory sizes
dir_sizes = {}

# Iterating through all the directories
for directory in filesystem:
    
    # Resetting the file size counter
    total_file_size = 0

    # Looping over the filesystem to look for child directories
    for filename, file_size in files.items():
        
        # Incrementing the total file size if the current directory is a parent directory
        if filename.startswith(directory):
            total_file_size += file_size
            
    # Setting the directory to the new collective size with children
    dir_sizes[directory] = total_file_size

In [4]:
# Setting a variable to hold the grand total size of all directories under 100000
grand_total_size = 0

# Looping through the directories to add all under 100000
for dir_name, dir_size in dir_sizes.items():
    
    # Checking to see if the directory is under 100000
    if dir_size <= 100000:
        
        # Incrementing the grand total size appropriately
        grand_total_size += dir_size
        
# Printing out the final results
print(f'Total size of all directories under 100000: {grand_total_size}')

Total size of all directories under 100000: 1886043


## Part 2

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:

- Delete directory e, which would increase unused space by 584.
- Delete directory a, which would increase unused space by 94853.
- Delete directory d, which would increase unused space by 24933642.
- Delete directory /, which would increase unused space by 48381165.

Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

In [5]:
# Getting all the directory sizes from the directory sizes dictionary
directory_sizes = dir_sizes.values()

In [6]:
# Creating a list to hold only the sizes that are workable per the parameters
workable_sizes = []

# Getting the unused space on the directory
unused_space = 70000000 - dir_sizes['/']

# Iterating over the directionary sizes
for dir_size in directory_sizes:
    
    # Adding the directory size to the workable sizes list if the condition is met
    if unused_space + dir_size >= 30000000:
        workable_sizes.append(dir_size)
        
# Getting the minimum value of these workable sizes
min_size = min(workable_sizes)

# Printing the final result
print(f'Size of the smallest directory to delete: {min_size}')

Size of the smallest directory to delete: 3842121
